In [2]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [28]:
# Paso 1: Crear la carpeta 'articles'
import os

# Crear la carpeta si no existe
if not os.path.exists('articles'):
    os.makedirs('articles')

# Paso 2: Descargar el PDF
url = 'https://dit.upm.es/~fsaez/intl/capitulos/5%20-Reingenier%EDa%20_I_.pdf'  # Reemplaza con la URL del PDF
pdf_name = 'REINGENIERÍA.pdf'  # Cambia esto por el nombre que desees

# Usando wget para descargar el PDF
!wget -O articles/{pdf_name} {url}

--2024-10-03 15:14:23--  https://dit.upm.es/~fsaez/intl/capitulos/5%20-Reingenier%EDa%20_I_.pdf
Resolving dit.upm.es (dit.upm.es)... 138.4.2.56, 2001:720:1500:42:20f:1fff:fe8d:18ba
Connecting to dit.upm.es (dit.upm.es)|138.4.2.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176314 (172K) [application/pdf]
Saving to: ‘articles/REINGENIERÍA.pdf’

articles/REINGENIER 100%[===================>] 172.18K   237KB/s    in 0.7s    

2024-10-03 15:14:24 (237 KB/s) - ‘articles/REINGENIERÍA.pdf’ saved [176314/176314]



In [5]:
DOCUMENTS_PATH = "./articles"
MODEL_EMBEDDING_NAME = "BAAI/bge-small-en-v1.5"
VECTORDB_PATH = "./vectorDB"

In [27]:
def get_prompt_template_with_context(context, question):
  prompt_template_with_context = f"""
    Contexto:
    {context}
    Responde la siguiente pregunta usando el contexto.
    Pregunta:
    {question}
    [/INST]
  """
  return prompt_template_with_context

In [31]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex

# Lista de modelos para embedding en HF hub: https://huggingface.co/spaces/mteb/leaderboard
Settings.embed_model = HuggingFaceEmbedding(model_name=MODEL_EMBEDDING_NAME)
Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

documents = SimpleDirectoryReader(DOCUMENTS_PATH).load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=True)

persist_dir = VECTORDB_PATH
index.storage_context.persist(persist_dir=persist_dir)

LLM is explicitly disabled. Using MockLLM.


Parsing nodes:   0%|          | 0/16 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/68 [00:00<?, ?it/s]

In [35]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine

class MyRetriever():

  query_engine = None
  top_k = 3

  def __init__(self):

    Settings.embed_model = HuggingFaceEmbedding(model_name=MODEL_EMBEDDING_NAME)
    Settings.llm = None
    Settings.chunk_size = 256
    Settings.chunk_overlap = 25

    index = self.load_index()

    # set number of docs to retreive
    top_k = 3

    # configure retriever
    retriever = VectorIndexRetriever(
      index=index,
      similarity_top_k=top_k,
    )

    # assemble query engine
    self.query_engine = RetrieverQueryEngine(
      retriever=retriever,
      node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
    )

  def load_index(self):
    storage_context = StorageContext.from_defaults(
      docstore=SimpleDocumentStore.from_persist_dir(persist_dir=VECTORDB_PATH),
      vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir=VECTORDB_PATH
      ),
      index_store=SimpleIndexStore.from_persist_dir(persist_dir=VECTORDB_PATH),
    )
    index = load_index_from_storage(storage_context)
    return index

  def get_contexts(self, query):
    return self.query_engine.query(query)

  def get_context(self, query):
    # reformat response
    context = "Context:\n"
    response = self.get_contexts(query)
    for i in range(self.top_k):
        context = context + response.source_nodes[i].text + "\n\n"
    return context

In [33]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

class MyLLM():

  tokenizer = None
  model = None

  def __init__(self):
    model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                device_map="auto",
                                                trust_remote_code=False,
                                                revision="main")

    config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
    model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

    self.model = model
    self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    self.model.eval()

  def get_response(self, prompt):
    inputs = self.get_inputs(prompt)
    outputs = self.model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
    response = self.tokenizer.batch_decode(outputs)[0]
    return response

  def get_tokenizer(self):
    return self.tokenizer

  def get_model(self):
    return self.model

  def get_inputs(self, prompt):
    inputs = self.tokenizer(prompt, return_tensors="pt")
    return inputs

In [36]:
class MyRAG:
  def __init__(self):
    self.retriever = MyRetriever()
    self.llm = MyLLM()

  def get_answer(self, question):
    prompt = self.get_prompt(question)
    return self.llm.get_response(prompt)

  def get_prompt(self, question):
    context = self.retriever.get_context(question)
    # prompt (with context)
    prompt = get_prompt_template_with_context(context, question)
    return prompt

In [37]:
rag = MyRAG()

LLM is explicitly disabled. Using MockLLM.


Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

In [38]:
question = "Que es la reingenieria"
print(rag.get_answer(question))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
    Contexto:
    Context:
Este último extrae 
ideas de J. Schuldt (1998) y E. Navarro. 
 
• El epígrafe “Lo que no es Reingenier ía” está basado en Hammer y Champy 
(1994).

Hammer y Champy: 
 
• “Reingeniería es la revisión fundam ental y el rediseño radical de 
procesos para alcanzar mejoras es pectaculares en medidas críticas y 
contemporáneas de rendimiento, tales como costos, calidad, servicio y 
rapidez”. 
 
 Profundizando en esta definición observ amos que contiene cuatro conceptos 
claves: 
1. Fundamental: Una vez se ha decidido pr oceder con la Reingeniería en un 
negocio, el individuo debe ha cerse las preguntas más básicas sobre su empresa y su 
funcionamiento. ¿Por qué hacemos las cosas de esta manera?, ¿No hay una forma mejor de hacerlas?. Estas preguntas obligan al empresario a cuestionar los supuestos más básicos sobre los que se asienta su negocio. Se lleva a cabo una revisión de todas las 
normas preestablecidas, que hasta el moment o eran incuestionables.

Por t